In [1]:
import os
import sys
import subprocess
import signac
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from statistics import linear_regression
from scipy.stats import linregress
from pymser import pymser

In [129]:
def is_slope_zero(n, simulations_per_block, threshold=0.03):
    # Step 1: Divide the Data
    n_length = len(n)
    # print(f"Total number of data points: {n_length}")

    # Calculate the number of blocks
    m = n_length // simulations_per_block
    # print(f"Number of blocks: {m}")

    # Initialize averages list
    averages = []

    for i in range(m):
        start_index = i * simulations_per_block
        # Handle the last block to include all remaining elements
        end_index = n_length if i == m - 1 else (i + 1) * simulations_per_block
        # print(f"Block {i + 1}: Start index = {start_index}, End index = {end_index}")

        # Calculate the average of the current block
        section_average = np.mean(n[start_index:end_index])
        averages.append(section_average)

    # Step 2: Scale the Averages between 0 and 1
    scaler = StandardScaler()  # You can change to MinMaxScaler() if desired
    scaled_averages = scaler.fit_transform(np.array(averages).reshape(-1, 1)).flatten()

    # Step 3: Divide into 5 groups
    group_size = m // 5
    slopes = []
    intercepts = []
    
    #Make the first 3 1/5 and the last 2 2/5
    for j in range(5):
        group_start = j * group_size
        group_end = (j + 1) * group_size if j < 3 else m  # Last group gets any remaining blocks
        # print(f"Group {j + 1}: Start = {group_start}, End = {group_end}")
        if group_end - group_start < 2:  # Need at least two points to calculate slope
            print(f"Group {j + 1} has insufficient data points.")
            continue
        
        x_group = np.arange(group_start, group_end)
        y_group = scaled_averages[group_start:group_end]
        slope, intercept, r_value, p_value, std_err = linregress(x_group, y_group)
        slopes.append(slope)
        intercepts.append(intercept)
        # print(f"Group {j + 1}: Slope = {slope}, Intercept = {intercept}")

    # print(f"Slopes: {slopes}")   
    # Step 3: Fit a Line with intercept fixed at 0
    x = np.arange(m)
    slope, intercept, r_value, p_value, std_err = linregress(x, scaled_averages)
    
    # Step 4: Check the Slope
    # print(slope, slopes[-1], threshold)
    equilibrated = (abs(slope) < threshold and abs(slopes[-1]) < threshold) or abs(slopes[-2]) < threshold*(2/3)

    if equilibrated:
        # Plotting
        plt.figure(figsize=(10, 6))

        # Original Data
        plt.subplot(1, 2, 1)
        plt.plot(scaler.transform(n.reshape(-1, 1)), label='Original Data', color='blue')
        
        # Draw vertical lines for each block average
        for i in range(m):
            # Calculate the x position scaled based on total number of data
            x_position = (i * simulations_per_block + (simulations_per_block / 2))  # Middle of the block
            plt.axvline(x=x_position, color='orange', linestyle='--', lw=0.7,
                        label=f'Block {i + 1} Average' if i == 0 else "")
        
        for j in range(1, 4):  # Draw lines after each of the first two groups
            group_boundary_x = j * group_size * simulations_per_block - (simulations_per_block / 2)
            plt.axvline(x=group_boundary_x, color='black', linestyle='-', lw=2,
                        label=f'Group {j} Boundary' if j == 1 else "")
        plt.title('Original Data')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.legend()

        # Averages and Line of Best Fit
        plt.subplot(1, 2, 2)
        plt.scatter(x, scaled_averages, label='Block Averages', color='orange')
        
        # Plot the line of best fit with intercept fixed at 0
        plt.plot(x, slope * x + intercept, label='Line of Best Fit (Intercept = 0)', color='red')
        plt.title('Block Averages and Line of Best Fit')
        plt.xlabel('Block')
        plt.ylabel('Scaled Average')
        plt.axhline(y=0, color='k', linestyle='--', lw=0.7)  # y=0 line for reference
        plt.legend()

        plt.tight_layout()
        plt.show()
    
    return slope, slopes, equilibrated

In [ ]:
# Find all jobs with the specified statepoint value
mol_name = "R170"
project = signac.get_project("opt_ff_ms")
jobs = project.find_jobs({"mol_name": mol_name})# "T": float(sys.argv[3]), "atom_type": int(sys.argv[2])})

# Iterate over the matching jobs
for job in jobs:
    # Construct the command using the job ID and statepoint value
    print("ID", job.id, "AT", job.sp.atom_type, "T", job.sp.T)
    try:
        df_box1 = np.genfromtxt(job.fn("gemc.eq.out.box2.prp"))
        energy = df_box1[:, 2 - 1]

        
        slope, group_slopes, result = is_slope_zero(energy, 15)
        print(f"The slope of all {round(slope,4)} and last 1/5 {round(group_slopes[-1],4)} or last 2/5 {round(group_slopes[-2],4)} of data is approximately zero: {result}")
    except:
        pass

In [ ]:
# #From Barnabas. Only seems to work on data that is long enough to equilibrate

# def running_average(data_raw):
#   n = data_raw.shape[0]
#   m = data_raw.shape[1]
#   running_ave = np.zeros([n, m])
#   for i in range(m):
#     for j in range(n):
#       mov_average = data_raw[0:j+1,i].mean()
#       running_ave[j,i] = mov_average
#   return(running_ave)


# def relative_abs_error(a,b):
#   error = np.abs(a-b)
#   rae = np.abs(error/a)
#   return rae

# import numpy as np

# def relative_abs_error(a, b):
#     """Calculate the relative absolute error between two values."""
#     return abs(a - b) / max(abs(a), abs(b))

# def test_equilibration(running_average_data):
#   start_index_frac = 0.60
#   stop_index_frac = 0.99
#   test_index_array = np.arange(start_index_frac, stop_index_frac, 0.005)
#   test_index = test_index_array*len(running_average_data)
#   test_points_index = test_index.astype(int)
#   equilibration_point = 0
#   for i in range(len(test_points_index)):
#     check_index = test_points_index[i]
#     equilibration_point = check_index
#     check_range_len = ((1 - stop_index_frac) * len(running_average_data)) - 1
#     check_range = np.arange(1, check_range_len, 1)
#     check_range = check_range.astype(int)
#     error_found = False

#     for k in (check_range):
#         rae = relative_abs_error(running_average_data[check_index], running_average_data[(check_index+k)])
#         if rae > 5e-4:
#             error_found = True  # Set the flag if an error is found
#             break  # Exit the inner loop if an error is found
#         else:
#           continue
#     if error_found == False:
#         break  # Exit the outer loop if no error is found

#   equilibration_time_index = equilibration_point
#   rem = equilibration_time_index%5
#   to_add = 0
#   if rem != 0:
#     to_add = 5 - rem
#   final_equib_time = equilibration_time_index + to_add
#   final_equib_time = int(final_equib_time)
#   final_equib_time_stored = np.array([final_equib_time])*1000
#   return final_equib_time_stored, equilibration_point, test_points_index


# mol_name = "R14"
# project = signac.get_project("opt_ff_ms")
# jobs = project.find_jobs({"mol_name": mol_name})# "T": float(sys.argv[3]), "atom_type": int(sys.argv[2])})

# # Iterate over the matching jobs
# for job in jobs:
#     # Construct the command using the job ID and statepoint value
#     print("ID", job.id, "AT", job.sp.atom_type, "T", job.sp.T)

#     # try:
#     df_box1 = np.genfromtxt(job.fn("gemc.eq.out.box2.prp"))
#     eq_col = df_box1[:, 2 - 1]
#     running_average_data = running_average(eq_col.reshape(-1,1))
#     # print(running_average_data)
#     results = test_equilibration(running_average_data)
#     final_equib_time_stored, equilibration_point, test_points_index = results
    
#     print("Equilibration point: ", equilibration_point)
#     fig, ax1 = plt.subplots(1, 1)

#     ax1.set_ylabel("Energy", color="black", fontsize=14, fontweight='bold')
#     ax1.set_xlabel("GEMC step", fontsize=14, fontweight='bold')

#     ax1.plot(range(len(eq_col)), 
#             eq_col, 
#             label = 'Raw data', 
#             color='blue')

#     ax1.plot(range(len(eq_col))[equilibration_point:], 
#             eq_col[equilibration_point:], 
#             label = 'Equilibrated data', 
#             color='red')

In [183]:
def plot_res_pymser(eq_col, results):
    fig, [ax1, ax2] = plt.subplots(1, 2, gridspec_kw={'width_ratios': [2, 1]}, sharey=True)

    ax1.set_ylabel("Energy", color="black", fontsize=14, fontweight='bold')
    ax1.set_xlabel("GEMC step", fontsize=14, fontweight='bold')

    ax1.plot(range(len(eq_col)), 
            eq_col, 
            label = 'Raw data', 
            color='blue')

    ax1.plot(range(len(eq_col))[results['t0']:], 
            results['equilibrated'], 
            label = 'Equilibrated data', 
            color='red')

    ax1.plot([0, len(eq_col)], 
            [results['average'], results['average']], 
            color='green', zorder=4, 
            label='Equilibrated average')

    ax1.fill_between(range(len(eq_col)), 
                    results['average'] - results['uncertainty'], 
                    results['average'] + results['uncertainty'], 
                    color='lightgreen', alpha=0.3, zorder=4)

    ax1.set_yticks(np.arange(0, eq_col.max()*1.1, eq_col.max()/10))
    ax1.set_xlim(-len(eq_col)*0.02, len(eq_col)*1.02)
    ax1.tick_params(axis="y", labelcolor="black")

    ax1.grid(alpha=0.3)
    ax1.legend()

    ax2.hist(eq_col, 
            orientation=u'horizontal', 
            bins=30, 
            edgecolor='blue', 
            lw=1.5, 
            facecolor='white', 
            zorder=3)

    ax2.hist(results['equilibrated'], 
            orientation=u'horizontal', 
            bins=3, 
            edgecolor='red', 
            lw=1.5, 
            facecolor='white', 
            zorder=3)

    ymax = int(ax2.get_xlim()[-1])

    ax2.plot([0, ymax], 
            [results['average'], results['average']],
            color='green', zorder=4, label='Equilibrated average')

    ax2.fill_between(range(ymax), 
                    results['average'] - results['uncertainty'],
                    results['average'] + results['uncertainty'],
                    color='lightgreen', alpha=0.3, zorder=4)

    ax2.set_xlim(0, ymax)

    ax2.grid(alpha=0.5, zorder=1)

    fig.set_size_inches(9,5)
    fig.set_dpi(100)
    fig.tight_layout()
    #fig.savefig('MSER_original.png', dpi=300, facecolor='white')
    plt.show()

In [ ]:
mol_name = "R170"
project = signac.get_project("opt_ff_ms")
jobs = project.find_jobs({"mol_name": mol_name})# "T": float(sys.argv[3]), "atom_type": int(sys.argv[2])})

# Iterate over the matching jobs
for job in jobs:
    # Construct the command using the job ID and statepoint value
    print("ID", job.id, "AT", job.sp.atom_type, "T", job.sp.T)

    try:
        df_box1 = np.genfromtxt(job.fn("gemc.eq.out.box2.prp"))
        eq_col = df_box1[:, 5 - 1]
        batch_size = int(np.maximum(1, int(len(eq_col)*0.0005)))
        # print("Batch size: ", batch_size)
        results = pymser.equilibrate(eq_col, LLM=False, batch_size=batch_size, ADF_test=True, uncertainty='uSD', print_results=False)
        
        equilibrated = len(eq_col) - results['t0'] > len(eq_col)*2/5
        if results["critical_values"]["1%"] < results["adf"]:
            equilibrated = False

        if equilibrated:
            log_text = '==============================================================================\n'
            plot_res_pymser(eq_col, results)
            print("       > Success! Found {} production cycles. Analyzing final data...\n\n".format(
                len(eq_col) - results['t0']))
            

        else:
            log_text = '==============================================================================\n'
            # plot_res_pymser(eq_col, results)
            print("       > Failure! Found {} production cycles. \n\n".format(
                len(eq_col) - results['t0']))
        print(results["adf"], results["critical_values"])
    except:
        pass